In [29]:
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime
import time
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from math import floor, ceil
from pylab import rcParams
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

%matplotlib inline

In [30]:
sns.set(style='ticks', palette='Spectral', font_scale=1.5)

material_palette = ["#4CAF50", "#2196F3", "#9E9E9E", "#FF9800", "#607D8B", "#9C27B0"]
sns.set_palette(material_palette)
rcParams['figure.figsize'] = 16, 8

plt.xkcd();
random_state = 42
np.random.seed(random_state)
tf.set_random_seed(random_state)

In [31]:
engine = sqlite3.connect('DB')
train_x = pd.read_sql("""
SELECT *
FROM train_x
""", engine)
train_y = pd.read_sql("""
SELECT *
FROM train_y
""", engine)

train_size = 0.8
train_cnt = floor(train_x.shape[0] * train_size)
x_train = train_x.iloc[0: train_cnt].values
y_train = train_y.iloc[0: train_cnt].values
x_test = train_x.iloc[train_cnt:].values
y_test = train_y.iloc[train_cnt:].values

In [32]:
print(len(train_y))
print(len(y_test))
print(train_cnt)

97056
19412
77644


In [33]:
def multilayer_perceptron(x, weights, biases, keep_prob):
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_1 = tf.nn.dropout(layer_1, keep_prob)
    out_layer = tf.matmul(layer_1, weights['out']) + biases['out']
    return out_layer

In [34]:
n_hidden_1 = 38
n_input = train_x.shape[1]#15 for 15 columns(the features)
n_classes = train_y.shape[1]#1 for 1 column(the labels)

weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'out': tf.Variable(tf.random_normal([n_hidden_1, n_classes]))
}

biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

keep_prob = tf.placeholder('float')

In [35]:
training_epochs = 50
display_step = 10
batch_size = 32

x = tf.placeholder('float', [None, n_input])
y = tf.placeholder('float', [None, n_classes])

In [36]:
predictions = multilayer_perceptron(x, weights, biases, keep_prob)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=predictions, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

In [37]:
print(len(train_x.columns))

17


In [38]:
# with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
    
#     for epoch in range(training_epochs):
#         avg_cost = 0.0
#         total_batch = int(len(x_train) / batch_size)
#         x_batches = np.array_split(x_train, total_batch)
#         y_batches = np.array_split(y_train, total_batch)
#         for i in range(total_batch):
#             batch_x, batch_y = x_batches[i], y_batches[i]
#             _, c = sess.run([optimizer, cost],
#                            feed_dict = {
#                                x: batch_x,
#                                y: batch_y,
#                                keep_prob: 0.8
#                            })
#             avg_cost += c / total_batch
#         if epoch % display_step == 0:
#             print("Epoch:", '%04d' % (epoch+1), "cost=", \
#                 "{:.9f}".format(avg_cost))
#     print('Done')
#     correct_prediction = tf.equal(tf.argmax(predictions, 1), tf.argmax(y, 1))
#     accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
#     print("Accuracy:", accuracy.eval({x: x_test, y: y_test, keep_prob: 1.0}))

In [46]:
def baseline_model():
    model = Sequential()
    model.add(Dense(20, input_dim = 17, kernel_initializer = 'normal', activation = 'relu'))
    model.add(Dense(3, activation = 'relu'))
    model.add(Dense(1, kernel_initializer = 'normal'))
    
    model.compile(loss = 'hinge', optimizer = 'adam')
    return model

In [40]:
"""
input_dim is number of columns
model.add(Dense(16, input_dim = 16, kernel_initializer = 'normal', activation = 'relu'))
That means it the layer has 16 nodes and expects 16 features(input variables)

model.add(Dense(8, activation='relu'))
This layer has 8 neurons

model.add(Dense(1, activation='sigmoid'))
This layer 1 neuron which you apply the sigmoid function to get an output

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
To compile the model

model.fit(X, Y, epochs=150, batch_size=10)
#Fit model

scores = model.evaluate(test_X, test_Y)
#Test model

"""

"\ninput_dim is number of columns\nmodel.add(Dense(16, input_dim = 16, kernel_initializer = 'normal', activation = 'relu'))\nThat means it the layer has 16 nodes and expects 16 features(input variables)\n\nmodel.add(Dense(8, activation='relu'))\nThis layer has 8 neurons\n\nmodel.add(Dense(1, activation='sigmoid'))\nThis layer 1 neuron which you apply the sigmoid function to get an output\n\nmodel.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])\nTo compile the model\n\nmodel.fit(X, Y, epochs=150, batch_size=10)\n#Fit model\n\nscores = model.evaluate(test_X, test_Y)\n#Test model\n\n"

In [41]:
print(len(x_train))
print(len(y_train))
print(len(x_test))
print(len(y_test))

77644
77644
19412
19412


In [47]:
# seed = 7
# np.random.seed(seed)
# estimator = KerasRegressor(build_fn=baseline_model, epochs=20, batch_size=250, verbose=0)
# kfold = KFold(n_splits = 10, random_state = seed)
# results = cross_val_score(estimator, x_train, y_train, cv=kfold)
# print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

trainedModel = baseline_model()
trainedModel.fit(x_train, y_train, validation_data = (x_test, y_test), epochs = 20, batch_size = 250)

Train on 77644 samples, validate on 19412 samples
Epoch 1/20
77644/77644 [==============================] - 1s 16us/step - loss: 351407.1713 - val_loss: 91.9009
Epoch 2/20
77644/77644 [==============================] - 0s 6us/step - loss: 114.1683 - val_loss: 91.5366